<a href="https://colab.research.google.com/github/marco-siino/DA-ESWA/blob/main/new_code/iss/Augmentation_ISS_JA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [1]:
import os
import os.path
from os import path
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
from urllib import request
!pip install -U deep-translator
from deep_translator import GoogleTranslator
from io import open
from pathlib import Path
from google.colab import files
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import class Dataset
module_url = f"https://raw.githubusercontent.com/marco-siino/DA-ESWA/main/code/dataset.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from dataset import Dataset

# Import class Vectorizer
module_url = f"https://raw.githubusercontent.com/marco-siino/DA-BT/main/code/vectorizer.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from vectorizer import Vectorizer

# Import class Simulator
module_url = f"https://raw.githubusercontent.com/marco-siino/DA-ESWA/main/code/simulator.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from simulator import Simulator

Fetching https://raw.githubusercontent.com/marco-siino/DA-ESWA/main/code/dataset.py
Fetching https://raw.githubusercontent.com/marco-siino/DA-BT/main/code/vectorizer.py
Fetching https://raw.githubusercontent.com/marco-siino/DA-ESWA/main/code/simulator.py


In [3]:
def chunkstring(string, length):
  res = list((string[0+i:length+i] for i in range(0, len(string), length)))
  return res

def enhance_one_sample(sample, TARGET='it', return_both=True):  
  preprocessed_text = Vectorizer.clean_samples(sample)
  #convert to a string 
  preprocessed_text = preprocessed_text.numpy().decode("utf-8") #numpy() --> bytes than --> decode() --> string  
  #print(type(preprocessed_text), preprocessed_text)
  #print(preprocessed_text)
  if not TARGET == 'mix':
    l_merged = ''
    #chunk to avoid character limits  
    TOBETRANS = chunkstring(preprocessed_text, 3000)
    translated_it = GoogleTranslator(source='en', target=TARGET).translate_batch(TOBETRANS)
    reversed_trans = GoogleTranslator(source=TARGET, target='en').translate_batch(translated_it)
    merged_chunks =''.join(reversed_trans)
    enhanced_sample = preprocessed_text+merged_chunks
    if return_both == False:
      enhanced_sample = merged_chunks
  
  elif TARGET == 'mix':
    
    l_merged = preprocessed_text   #first original sample preprocessed
    TOBETRANS = chunkstring(preprocessed_text, 3000)  #batch to fit googletrans limit
    
    #  ### ITALIAN   ###
    translated_it = GoogleTranslator(source='en', target='it').translate_batch(TOBETRANS, return_both=False )
    reversed_trans = GoogleTranslator(source='it', target='en').translate_batch(translated_it)
    merged_chunks =''.join(reversed_trans)
    l_merged = l_merged + merged_chunks 
    #print('IT  ok')

    #  ###  GERMAN  ###
    translated_de = GoogleTranslator(source='en', target='de').translate_batch(TOBETRANS, return_both=False)
    reversed_trans = GoogleTranslator(source='de', target='en').translate_batch(translated_de)
    merged_chunks =''.join(reversed_trans)
    l_merged = l_merged + merged_chunks
    #print('DE  ok')

    #  ###  TURKISH  ###
    translated_tr = GoogleTranslator(source='en', target='tr').translate_batch(TOBETRANS, return_both=False)
    reversed_trans = GoogleTranslator(source='tr', target='en').translate_batch(translated_tr)
    merged_chunks =''.join(reversed_trans)
    l_merged = l_merged + merged_chunks
    #print('TR  ok')

    #  ###  JAPANESE  ###
    translated_ja = GoogleTranslator(source='en', target='ja').translate_batch(TOBETRANS, return_both=False)
    reversed_trans = GoogleTranslator(source='ja', target='en').translate_batch(translated_ja)
    merged_chunks =''.join(reversed_trans)
    l_merged = l_merged + merged_chunks
    #print('JA  ok')

    enhanced_sample =  l_merged 
  return enhanced_sample

def select_ds(dict_url_names, ds):
  if  ds == 'fns':
    dict_url_names = {k:dict_url_names[k] for k in dict_url_names.keys() if 'fns' in k}
  elif ds == 'hss':
    dict_url_names = {k:dict_url_names[k] for k in dict_url_names.keys() if 'hss' in k}
  elif  ds == 'iss':
    dict_url_names = {k:dict_url_names[k] for k in dict_url_names.keys() if 'iss' in k}
  else:
    print('no correct ds')
  return dict_url_names

# Read and download

In [4]:
langs_list = GoogleTranslator().get_supported_languages()  # output: [arabic, french, english etc...]

langs_dict = GoogleTranslator().get_supported_languages(as_dict=True)  # output: {arabic: ar, french: fr, english:en etc...}
print('available langs_dict', langs_dict)

available langs_dict {'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'assamese': 'as', 'aymara': 'ay', 'azerbaijani': 'az', 'bambara': 'bm', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bhojpuri': 'bho', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-CN', 'chinese (traditional)': 'zh-TW', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dhivehi': 'dv', 'dogri': 'doi', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'ewe': 'ee', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'guarani': 'gn', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'iw', 'hindi': 'hi', 'hmong': 'hmn', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'ilocano': 'ilo', 'indonesian': 'id', 'irish': 'ga', 'italian': 'it', 'japanese':

In [5]:
link_drive_to_be_github = '/content/drive/MyDrive/DS_Names.xlsx'

In [6]:
df = pd.read_excel(link_drive_to_be_github)
#print(df)
example_url = pd.read_excel(link_drive_to_be_github)['URL'][0]
#urls =pd.read_excel(link_drive_to_be_github)['URL'].to_list()
dict_url_names =dict(zip(df['URL'].to_list() , df['subfolder name'].to_list()))
dict_path_truth =dict(zip(df['URL'].to_list() , df['truth_file_path'].to_list()))
print(dict_url_names)
print(dict_path_truth )

{'https://github.com/marco-siino/DA-ESWA/raw/main/data/fns/fns-training-original.zip': 'pan20-author-profiling-training-2020-02-23', 'https://github.com/marco-siino/DA-ESWA/raw/main/data/fns/fns-test-original.zip': 'pan20-author-profiling-test-2020-02-23', 'https://github.com/marco-siino/DA-ESWA/raw/main/data/hss/hss-training-original.zip': 'pan21-author-profiling-training-2021-03-14', 'https://github.com/marco-siino/DA-ESWA/raw/main/new_data/hss/hss-test-original.zip': 'pan21-author-profiling-test-without-gold', 'https://github.com/marco-siino/DA-ESWA/raw/main/data/iss/iss-training-original.zip': 'pan22-author-profiling-training-2022-03-29', 'https://github.com/marco-siino/DA-ESWA/raw/main/data/iss/iss-test-original.zip': 'pan22-author-profiling-test-2022-04-22-without_truth'}
{'https://github.com/marco-siino/DA-ESWA/raw/main/data/fns/fns-training-original.zip': 'pan20-author-profiling-training-2020-02-23/en/truth.txt', 'https://github.com/marco-siino/DA-ESWA/raw/main/data/fns/fns-tes

# Multi URLS

In [7]:
dict_url_names = select_ds(dict_url_names, 'iss')
urls = [*dict_url_names]

selcted_lang =  {#'turkish': 'tr',
                 'japanese': 'ja',
                 #'italian': 'it',
                 #'german': 'de',
                 #'mixed-it-de-tr-ja': 'mix'
                }

print('#####   Selected Dataset #### \n', urls )
print('#####   selcted_lang    #####\n', selcted_lang)


#coutner and time
start_time = time.time()
count_url = 0.0
tot_sample = 0.0

 
for url in urls: 
  print('url', url)
  #download xip file  from url
  train_set_url = url 
  train_set_path = tf.keras.utils.get_file(dict_url_names[url] + '.zip', train_set_url,
                                      extract=True, archive_format='zip',cache_dir='.',
                                      cache_subdir='')

  print('train_set_path',train_set_path) #print file location

  for key in [*selcted_lang]:  #loop over languagues
    lang = selcted_lang[key]
    print('##########    LANG   #########\t', lang)
    #remove augmented fro name
    if 'augmented' in dict_url_names[url]:
      l = dict_url_names[url].split('-')[:-1]
      cleaned_name = "-".join(l)
    else:
      cleaned_name = dict_url_names[url]  #get the name of the file
    #print('new_folder_name:::: ', new_folder_name)
    new_folder_name = cleaned_name +'-' + lang + '/'
    # Generate the augmented training set as a zip file.
    if  not os.path.exists(new_folder_name):
        os.makedirs(new_folder_name   )
        print('new_folder_name lang' , new_folder_name   )
        #move truth file
        FROM = dict_path_truth[url] 
        DEST = new_folder_name     + 'truth.txt'
        shutil.copyfile(FROM,DEST)
        print('truth file moved to \t', DEST)
    if not os.path.exists(new_folder_name + 'en' + '/'):
        os.makedirs(new_folder_name + 'en' + '/')
    ###############àà
    file_nr = 0
    count_errors = 0
    print('xml files in original folder::', os.listdir(dict_url_names[url] + '/en'))
    print('tot files', len(os.listdir(dict_url_names[url] + '/en')))

    #augment andove all xml files
    filelist = os.listdir(dict_url_names[url] + '/en/')
    filelist = [i for i in filelist if '.xml' in i]
    
    # ::::: AUGMENTATION  :::::::
    for filename in filelist:
      #update counters
      tot_sample +=1
      file_nr += 1
      
      x = filename.split(".") #clean name     
      author_id = x[0]
      print("File nr:::  ", file_nr, "  Filename:::", filename)
      text = open(dict_url_names[url] + '/en/' +filename, 'r').read() #open english sample so it's the original locaiton 
      
      #print only first sample
      #if file_nr == 1:  
      #preprocessed_sample = custom_standardization(text) #return only pre prcessed smaple 
      enhanced_sample = enhance_one_sample(text, 
                                            TARGET=lang)   #trnaslate each sample

      # print('\n\ncleaned and trans sample\n',
      #        enhanced_sample)
      #write augmente samples
      f = open( new_folder_name + "en/"+author_id+".xml", "a")
      f.write(enhanced_sample)
      f.close()
      #print('Succes')
    count_url += 1
    
    #create and download .zip file with augmented ds
    NAME_OUT = new_folder_name[:-1]
    zip_file_name = new_folder_name[:-1] + ".zip"
    print('NAME_OUT', NAME_OUT)  #last without .zip
    print('zip_file_name', zip_file_name) #with .zip
    
    
    print('path folder', os.getcwd() +'/'  + NAME_OUT)
  
    #create zip file
    shutil.make_archive(os.getcwd() +'/'  + NAME_OUT,
                        'zip',
                        os.getcwd(),
                        NAME_OUT)
    files.download(os.getcwd() +'/'  + NAME_OUT+ '.zip')  #only in colab

    print('number of files in folder:::', file_nr)
    print('\nEND DATASET lang', lang ,'######')
    print()
    print()
print('tot_url:::\t',count_url)
print('tot smaples::: \t', tot_sample)
print("--- %s seconds ---" % (time.time() - start_time))

#####   Selected Dataset #### 
 ['https://github.com/marco-siino/DA-ESWA/raw/main/data/iss/iss-training-original.zip', 'https://github.com/marco-siino/DA-ESWA/raw/main/data/iss/iss-test-original.zip']
#####   selcted_lang    #####
 {'japanese': 'ja'}
url https://github.com/marco-siino/DA-ESWA/raw/main/data/iss/iss-training-original.zip
5806456/5806456 [==============================] - 0s 0us/step
train_set_path ./pan22-author-profiling-training-2022-03-29.zip
##########    LANG   #########	 ja
new_folder_name lang pan22-author-profiling-training-2022-03-29-ja/
truth file moved to 	 pan22-author-profiling-training-2022-03-29-ja/truth.txt
xml files in original folder:: ['3f66669bee36a1ded41f43e62475ee72.xml', 'ad89cb1202d52f8cf0f6c7253774b9ad.xml', 'd9d352af2dc71dfbdc74cc13f11e7a48.xml', '8f228cf2ac478a9bc779c59e4b3c7a74.xml', '71ec2e3c7561877b3b79b3664fd5d877.xml', 'bd7c8f48dae016b03cfb8bc961bc3b4d.xml', '8a6bd8432f68a939c809a36c9ec80bf.xml', '35c40613f308a7c12e6f5e06307b43f6.xml', 'b9

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

number of files in folder::: 420

END DATASET lang ja ######


url https://github.com/marco-siino/DA-ESWA/raw/main/data/iss/iss-test-original.zip
2446477/2446477 [==============================] - 0s 0us/step
train_set_path ./pan22-author-profiling-test-2022-04-22-without_truth.zip
##########    LANG   #########	 ja
new_folder_name lang pan22-author-profiling-test-2022-04-22-without_truth-ja/
truth file moved to 	 pan22-author-profiling-test-2022-04-22-without_truth-ja/truth.txt
xml files in original folder:: ['2512304c7a847489ea0e561dd34df86a.xml', '199b3fb39de3ef20bda586a7a7ef8932.xml', '7d152fc274395994032a5cff4d5dbba6.xml', '8a030f3bbfcf12db415cd1b19c69dbd8.xml', 'aff307c6397a566f998178f78fc516a2.xml', '35206cb65f916c8390d9ef7a31cb6e01.xml', 'ba5c2b0d20b249842b210ccdb6b4c3e4.xml', 'ffc409cdad92bf2d15f4a9e18b250b31.xml', '9228dac155633b13780552bc01dc2e0.xml', 'c32bf7ecce95dd084f17e76f5c3d11c2.xml', '18cab06282ac3225d75b2cdafde1520b.xml', 'd53275e89bd24a8e6de31555f4d1bffe.xml', 'd924

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

number of files in folder::: 180

END DATASET lang ja ######


tot_url:::	 2.0
tot smaples::: 	 600.0
--- 12353.700949430466 seconds ---


In [ ]:
# started at 14.09